Setting up Huggingface Neuralcoref and small example

In [1]:
# !pip install spacy==2.2.4
# compile neuralcoref from scratch according to https://github.com/huggingface/neuralcoref

In [2]:
import spacy
print(spacy.__version__) 

2.2.4


In [3]:
import en_core_web_lg
# Add neural coref to SpaCy's pipe
import neuralcoref

nlp = en_core_web_lg.load()
neuralcoref.add_to_pipe(nlp)

In [18]:

doc = nlp("Eva and Martha didn't want their friend Jenny to feel lonely so they invited her to the party.")

In [19]:
#doc._.has_coref
print(doc._.coref_clusters)

[Eva and Martha: [Eva and Martha, their, they], Jenny: [Jenny, her]]


Setting up AllenNLP coreference resolution

In [6]:
#!pip install allennlp
#!pip install allennlp-models

In [7]:
from allennlp.predictors.predictor import Predictor

In [12]:
model_url = 'https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2020.02.27.tar.gz'
predictor = Predictor.from_path(model_url)  # load the model

In [13]:
text = "Eva and Martha didn't want their friend Jenny to feel lonely so they invited her to the party."
prediction = predictor.predict(document=text)  # get the prediction

In [14]:
def get_clusters_allen_nlp(prediction):
    clusters = []
    for cluster in prediction['clusters']:
        print("cluster", cluster)
        first_mention = ""
        mention_ref = {}
        for token in cluster:
            token_start = token[0]
            token_end = token[1]
            mention = " ".join(prediction['document'][token_start:token_end + 1])
            if first_mention == "":
                first_mention = mention
                mention_ref[first_mention] = [first_mention]
            else:
                mention_ref[first_mention] += [mention]
        print(mention_ref)
        clusters.append(mention_ref)
    return clusters


In [15]:
clusters_allennlp = get_clusters_allen_nlp(prediction)
print(clusters_allennlp)

cluster [[0, 2], [6, 6], [13, 13]]
{'Eva and Martha': ['Eva and Martha', 'their', 'they']}
cluster [[6, 8], [15, 15]]
{'their friend Jenny': ['their friend Jenny', 'her']}
[{'Eva and Martha': ['Eva and Martha', 'their', 'they']}, {'their friend Jenny': ['their friend Jenny', 'her']}]
